# Question One

In [22]:
using JuMP, Cbc
polls = [[80 34];
        [60 44];
        [40 44];
        [20 24];
        [40 114];
        [40 64];
        [70 14];
        [50 44];
        [70 54];
        [70 64]]
m = Model(solver = CbcSolver())

@variable(m, x[1:10, 1:5], Bin)
@variable(m, y[1:5], Bin)

for i in 1:10
    @constraint(m, sum(x[i,:]) == 1)
end

# constraint that ea. distrcit must contain btwn 150 and 250 voters (w/ no independo's)
for k in 1:5
    @constraint(m, 150 <= sum{(polls[i,1] + polls[i, 2]) * x[i,k], i=1:10} <= 250)
end

# constraint for handling the all or none restrictment
# if there is a majority (Demo) then z[i] = 1
# else we need to find where it leans
for k in 1:5
    @constraint(m, sum{(polls[i,2] - polls[i, 1]) * x[i,k], i=1:10} >= -250*(1-y[k]))
end

@objective(m, Max, sum(y))
status = solve(m)
println(status)

for j in 1:5
    
    println("District ", j)
    if(getvalue(y[j]) == 1)
        println("Democratic Majority")
    else
        println("Republican Majority")
    end
    
    println("Zone Contains: ")
    
    for i in 1:10
        if(getvalue(x[i,j]) >= 0.99)
            println("City: ", i)
        end      
    end
    println(" ")
end

println("Minimized objective value: ", getobjectivevalue(m))

Optimal
District 1
Republican Majority
Zone Contains: 
City: 1
City: 2
 
District 2
Republican Majority
Zone Contains: 
City: 7
City: 10
 
District 3
Democratic Majority
Zone Contains: 
City: 3
City: 4
City: 8
 
District 4
Democratic Majority
Zone Contains: 
City: 6
City: 9
 
District 5
Democratic Majority
Zone Contains: 
City: 5
 
Minimized objective value: 3.0
